# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 8 2022 8:45AM,245924,10,EYE16VFS27HWUM,Eye Pharma Inc,Released
1,Sep 8 2022 8:45AM,245924,10,EYE16VFQS6MSU,Eye Pharma Inc,Released
2,Sep 8 2022 8:44AM,245921,10,0085968688,ISDIN Corporation,Released
3,Sep 8 2022 8:44AM,245921,10,0085968691,ISDIN Corporation,Released
4,Sep 8 2022 8:44AM,245921,10,0085968690,ISDIN Corporation,Released
5,Sep 8 2022 8:44AM,245921,10,0085968695,ISDIN Corporation,Released
6,Sep 8 2022 8:44AM,245921,10,0085968696,ISDIN Corporation,Released
7,Sep 8 2022 8:44AM,245921,10,0085968697,ISDIN Corporation,Released
8,Sep 8 2022 8:44AM,245921,10,0085968725,ISDIN Corporation,Released
9,Sep 8 2022 8:44AM,245921,10,0085968726,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,245920,Released,1
49,245921,Released,23
50,245922,Released,1
51,245923,Released,2
52,245924,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245920,NaN,NaN,1.0
245921,NaN,NaN,23.0
245922,NaN,NaN,1.0
245923,NaN,NaN,2.0
245924,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245648,0.0,1.0,0.0
245649,0.0,0.0,1.0
245650,0.0,0.0,1.0
245714,25.0,1.0,0.0
245729,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245648,0,1,0
245649,0,0,1
245650,0,0,1
245714,25,1,0
245729,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245648,0,1,0
1,245649,0,0,1
2,245650,0,0,1
3,245714,25,1,0
4,245729,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245648,,1,
1,245649,,,1
2,245650,,,1
3,245714,25,1,
4,245729,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 8 2022 8:45AM,245924,10,Eye Pharma Inc
2,Sep 8 2022 8:44AM,245921,10,ISDIN Corporation
25,Sep 8 2022 8:43AM,245923,16,Sartorius Bioprocess Solutions
27,Sep 8 2022 8:39AM,245922,10,"Methapharm, Inc."
28,Sep 8 2022 8:32AM,245919,10,ISDIN Corporation
77,Sep 8 2022 8:31AM,245920,10,"Goodwin Biotechnology, Inc."
78,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC"
79,Sep 8 2022 8:30AM,245910,10,Eye Pharma Inc
80,Sep 8 2022 8:23AM,245918,16,"SHL Pharma, LLC"
81,Sep 8 2022 8:22AM,245917,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 8 2022 8:45AM,245924,10,Eye Pharma Inc,,,2
1,Sep 8 2022 8:44AM,245921,10,ISDIN Corporation,,,23
2,Sep 8 2022 8:43AM,245923,16,Sartorius Bioprocess Solutions,,,2
3,Sep 8 2022 8:39AM,245922,10,"Methapharm, Inc.",,,1
4,Sep 8 2022 8:32AM,245919,10,ISDIN Corporation,,,49
5,Sep 8 2022 8:31AM,245920,10,"Goodwin Biotechnology, Inc.",,,1
6,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC",,,1
7,Sep 8 2022 8:30AM,245910,10,Eye Pharma Inc,,,1
8,Sep 8 2022 8:23AM,245918,16,"SHL Pharma, LLC",,1,
9,Sep 8 2022 8:22AM,245917,10,ISDIN Corporation,,,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:45AM,245924,10,Eye Pharma Inc,2,,
1,Sep 8 2022 8:44AM,245921,10,ISDIN Corporation,23,,
2,Sep 8 2022 8:43AM,245923,16,Sartorius Bioprocess Solutions,2,,
3,Sep 8 2022 8:39AM,245922,10,"Methapharm, Inc.",1,,
4,Sep 8 2022 8:32AM,245919,10,ISDIN Corporation,49,,
5,Sep 8 2022 8:31AM,245920,10,"Goodwin Biotechnology, Inc.",1,,
6,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC",1,,
7,Sep 8 2022 8:30AM,245910,10,Eye Pharma Inc,1,,
8,Sep 8 2022 8:23AM,245918,16,"SHL Pharma, LLC",,1,
9,Sep 8 2022 8:22AM,245917,10,ISDIN Corporation,19,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:45AM,245924,10,Eye Pharma Inc,2,,
1,Sep 8 2022 8:44AM,245921,10,ISDIN Corporation,23,,
2,Sep 8 2022 8:43AM,245923,16,Sartorius Bioprocess Solutions,2,,
3,Sep 8 2022 8:39AM,245922,10,"Methapharm, Inc.",1,,
4,Sep 8 2022 8:32AM,245919,10,ISDIN Corporation,49,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:45AM,245924,10,Eye Pharma Inc,2.0,NaN,NaN
1,Sep 8 2022 8:44AM,245921,10,ISDIN Corporation,23.0,NaN,NaN
2,Sep 8 2022 8:43AM,245923,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
3,Sep 8 2022 8:39AM,245922,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Sep 8 2022 8:32AM,245919,10,ISDIN Corporation,49.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:45AM,245924,10,Eye Pharma Inc,2.0,0.0,0.0
1,Sep 8 2022 8:44AM,245921,10,ISDIN Corporation,23.0,0.0,0.0
2,Sep 8 2022 8:43AM,245923,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
3,Sep 8 2022 8:39AM,245922,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Sep 8 2022 8:32AM,245919,10,ISDIN Corporation,49.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3934118,150.0,0.0,22.0
15,737379,64.0,2.0,70.0
16,983584,8.0,1.0,0.0
19,1229536,12.0,4.0,1.0
20,1229390,16.0,16.0,12.0
21,1965786,4.0,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3934118,150.0,0.0,22.0
1,15,737379,64.0,2.0,70.0
2,16,983584,8.0,1.0,0.0
3,19,1229536,12.0,4.0,1.0
4,20,1229390,16.0,16.0,12.0
5,21,1965786,4.0,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,150.0,0.0,22.0
1,15,64.0,2.0,70.0
2,16,8.0,1.0,0.0
3,19,12.0,4.0,1.0
4,20,16.0,16.0,12.0
5,21,4.0,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,150.0
1,15,Released,64.0
2,16,Released,8.0
3,19,Released,12.0
4,20,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,22.0,70.0,0.0,1.0,12.0,0.0
Executing,0.0,2.0,1.0,4.0,16.0,4.0
Released,150.0,64.0,8.0,12.0,16.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,22.0,70.0,0.0,1.0,12.0,0.0
1,Executing,0.0,2.0,1.0,4.0,16.0,4.0
2,Released,150.0,64.0,8.0,12.0,16.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,22.0,70.0,0.0,1.0,12.0,0.0
1,Executing,0.0,2.0,1.0,4.0,16.0,4.0
2,Released,150.0,64.0,8.0,12.0,16.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()